### Test topographic features
One issue is that normalising aspect is not straight forward (0-360 degrees)

a suggestion here is to represent as cosine(deg) and sine(deg) - but potentially adding an extra param here is too much of a cost for model complexity
https://stats.stackexchange.com/questions/185091/normalisation-of-circular-statistics-such-as-wind-direction-in-degrees-for-clu

In [9]:
import ee
from geemap import geemap
ee.Initialize()

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [25]:
fp_train_ext = "/home/markdj/Dropbox/artio/polesia/val/Vegetation_extent_rough.shp"
aoi = geemap.shp_to_ee(fp_train_ext)

In [57]:
def fetch_topography(aoi):
    """
    Get static topographic layers 
    elevation, slope and aspect from the NASA SRTM product 30m ('USGS/SRTMGL1_003').
    Also gets 'Global SRTM Topographic Diversity' ('CSP/ERGo/1_0/Global/SRTM_topoDiversity'), 
    but I'm not convinced this is useful - 270m, looks a lot like coarse res slope/elevation

    :param aoi: ee.featurecollection.FeatureCollection, used to indicate AOI extent 
    :return: ee.image.Image, stack of composite images with bands: ['elevation', 'aspect', 'slope', 'topographic_diversity']     
    """
    print('fetch_topography(): hi!')
    #datasets & bands
    srtm = ee.Image('USGS/SRTMGL1_003')
    ds_topodiv = ee.Image('CSP/ERGo/1_0/Global/SRTM_topoDiversity')
    elevation = srtm.select('elevation').clip(aoi.geometry())
    topodiversity = ds_topodiv.select('constant').clip(aoi.geometry()).rename('topographic_diversity')
    
    # derive slope & aspect
    slope = ee.Terrain.slope(elevation)
    aspect = ee.Terrain.aspect(elevation)

    # compile
    stack = elevation.addBands(slope)
    stack = stack.addBands(aspect)
    stack = stack.addBands(topodiversity)
    print('fetch_topography(): bye!')
    return stack

In [52]:
def map_topography(stack, lat=51.85, lon=27.8, elv_rng=(100.0,200.0), asp_rng=(0.0,360.0), slp_rng=(0.0, 10.0), topdiv_rng=(0,0.15)):
    """
    Quick mapping function for debugging topo data 
    : param stack: Image, stack containing topographic composite images for AOI
    : param lat: float, map central latitude
    : param lon: float, map central longitude
    : params elv_rng, asp_rng, slp_rng, topdiv_rng: tuples of floats, min/max z-values for elevation, aspect, slope, and topo diversity, respectively. 
                                                    Defaults are suited to the Polesia training AOI    
    : return : 
    """
    Map = geemap.Map(center=(lat, lon), zoom=9)
    Map.add_basemap('SATELLITE')
    Map.addLayer(stack, {'min': elv_rng[0],'max': elv_rng[1], 'bands': ['elevation']}, 'elevation')
    Map.addLayer(stack, {'min': asp_rng[0],'max': asp_rng[1], 'bands': ['aspect']}, 'aspect')
    Map.addLayer(stack, {'min': slp_rng[0],'max': slp_rng[1], 'bands': ['slope']}, 'slope')
    Map.addLayer(stack, {'min': topdiv_rng[0],'max': topdiv_rng[1], 'bands': ['topographic_diversity']}, 'topographic_diversity')
    return Map

In [43]:
srtm = ee.Image('USGS/SRTMGL1_003')
ds_topodiv = ee.Image('CSP/ERGo/1_0/Global/SRTM_topoDiversity')
#srtm.getInfo()

In [53]:
elevation = srtm.select('elevation').clip(aoi.geometry())
topodiversity = ds_topodiv.select('constant').clip(aoi.geometry()).rename('topographic_diversity')

# derive slope & aspect
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

# compile
stack = elevation.addBands(slope)
stack = stack.addBands(aspect)
stack = stack.addBands(topodiversity)
#stack.getInfo()

Topographic diversity looks a lot like slope (but coarser res) to me! probably abandon?

In [51]:
lat=51.85
lon=27.8
Map = geemap.Map(center=(lat, lon), zoom=9)
Map.add_basemap('SATELLITE')
Map.addLayer(stack, {'min': 100.0,'max': 200, 'bands': ['elevation']}, 'elevation')
Map.addLayer(stack, {'min': 0,'max': 360, 'bands': ['aspect']}, 'aspect')
Map.addLayer(stack, {'min': 0.0,'max': 10.0, 'bands': ['slope']}, 'slope')
Map.addLayer(stack, {'min': 0.0,'max': 0.15, 'bands': ['topographic_diversity']}, 'topographic_diversity')
Map

Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [61]:
stack = fetch_topography(aoi)
map_topography(stack, elv_rng=(110,170))

fetch_topography(): hi!
fetch_topography(): bye!


Map(center=[51.85, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…